# Line3 Dataset Pipeline - GT Masks Visualization

This notebook examines the ground truth masks in the line3_dataset to verify they contain real data and visualize their content.

## Overview
- Load and examine .mat files containing ground truth masks
- Visualize the masks to verify they contain real data
- Check data statistics and integrity
- Compare different mouse datasets


In [7]:
# Import required libraries
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import h5py
from scipy.io import loadmat, savemat
from scipy import sparse
import glob
from pathlib import Path

# Set matplotlib backend for Jupyter
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
plt.ioff()  # Turn off interactive mode

# Clear any existing path modifications
for path in sys.path[:]:
    if 'suns' in path or 'Shallow-UNet' in path:
        sys.path.remove(path)

# Use absolute path to the correct config file
config_path = '/gpfs/data/shohamlab/nicole/code/SUNS_nicole/suns/config.py'

# Import the specific config file
import importlib.util
spec = importlib.util.spec_from_file_location("config", config_path)
config = importlib.util.module_from_spec(spec)
spec.loader.exec_module(config)

# Use the config variables
DATAFOLDER_SETS = config.DATAFOLDER_SETS
ACTIVE_EXP_SET = config.ACTIVE_EXP_SET
EXP_ID_SETS = config.EXP_ID_SETS

print(f"Config file loaded from: {config_path}")
print(f"Active dataset: {ACTIVE_EXP_SET}")
print(f"Data folder: {DATAFOLDER_SETS[ACTIVE_EXP_SET]}")
print(f"Experiment IDs: {EXP_ID_SETS[ACTIVE_EXP_SET]}")

# Verify we're using line3_dataset
if ACTIVE_EXP_SET == 'line3_dataset':
    print("✓ Successfully loaded line3_dataset configuration")
else:
    print(f"⚠ Warning: Expected line3_dataset but got {ACTIVE_EXP_SET}")
    print("This suggests the wrong config file is being loaded.")


importing config
Config file loaded from: /gpfs/data/shohamlab/nicole/code/SUNS_nicole/suns/config.py
Active dataset: line3_dataset
Data folder: /gpfs/data/shohamlab/nicole/code/SUNS_nicole/demo/line3_dataset
Experiment IDs: ['mouse6', 'mouse7', 'mouse10', 'mouse12']
✓ Successfully loaded line3_dataset configuration


## 1. Load and Examine GT Mask Files

First, let's load the ground truth mask files and examine their content.


In [8]:
# Set up paths
data_folder = DATAFOLDER_SETS[ACTIVE_EXP_SET]
dir_Masks = os.path.join(data_folder, 'GT Masks')
exp_ids = EXP_ID_SETS[ACTIVE_EXP_SET]

print(f"GT Masks directory: {dir_Masks}")
print(f"Experiment IDs: {exp_ids}")

# Check if directory exists
if not os.path.exists(dir_Masks):
    print(f"Error: GT Masks directory not found: {dir_Masks}")
else:
    print(f"✓ GT Masks directory found")
    
    # List all .mat files
    mat_files = glob.glob(os.path.join(dir_Masks, '*.mat'))
    print(f"Found {len(mat_files)} .mat files:")
    for f in sorted(mat_files):
        file_size = os.path.getsize(f) / (1024*1024)  # MB
        print(f"  {os.path.basename(f)}: {file_size:.2f} MB")


GT Masks directory: /gpfs/data/shohamlab/nicole/code/SUNS_nicole/demo/line3_dataset/GT Masks
Experiment IDs: ['mouse6', 'mouse7', 'mouse10', 'mouse12']
✓ GT Masks directory found
Found 8 .mat files:
  FinalMasks_mouse10.mat: 38.19 MB
  FinalMasks_mouse10_sparse.mat: 0.05 MB
  FinalMasks_mouse12.mat: 28.38 MB
  FinalMasks_mouse12_sparse.mat: 0.03 MB
  FinalMasks_mouse6.mat: 60.50 MB
  FinalMasks_mouse6_sparse.mat: 0.06 MB
  FinalMasks_mouse7.mat: 69.00 MB
  FinalMasks_mouse7_sparse.mat: 0.08 MB


## 2.5. Sparse Matrix Shape and Construction Analysis

Let's examine the shape and construction of the sparse matrices to verify they contain non-zero values only where there are neurons (ROIs).


In [12]:
def analyze_sparse_matrix_structure(sparse_masks, exp_id, original_dense_masks=None):
    """Analyze the structure and construction of sparse matrices."""
    print(f"\n=== Sparse Matrix Structure Analysis for {exp_id} ===")
    
    if sparse_masks is None:
        print("❌ No sparse matrix to analyze")
        return
    
    # Basic properties
    print(f"Basic Properties:")
    print(f"  Type: {type(sparse_masks)}")
    print(f"  Format: {sparse_masks.format}")
    print(f"  Shape: {sparse_masks.shape}")
    print(f"  Data type: {sparse_masks.dtype}")
    print(f"  Number of non-zero elements: {sparse_masks.nnz:,}")
    print(f"  Total elements: {sparse_masks.size:,}")
    print(f"  Sparsity: {1 - (sparse_masks.nnz / sparse_masks.size):.4f}")
    
    # Memory usage
    if hasattr(sparse_masks, 'data'):
        memory_usage = sparse_masks.data.nbytes + sparse_masks.indices.nbytes + sparse_masks.indptr.nbytes
        dense_memory = sparse_masks.size * sparse_masks.dtype.itemsize
        compression_ratio = memory_usage / dense_memory
        
        print(f"\nMemory Usage:")
        print(f"  Sparse memory: {memory_usage / (1024*1024):.2f} MB")
        print(f"  Dense memory: {dense_memory / (1024*1024):.2f} MB")
        print(f"  Compression ratio: {compression_ratio:.2%}")
        print(f"  Space saved: {(1 - compression_ratio):.1%}")
    
    # Analyze data distribution
    if hasattr(sparse_masks, 'data'):
        print(f"\nData Distribution:")
        print(f"  Min value: {np.min(sparse_masks.data)}")
        print(f"  Max value: {np.max(sparse_masks.data)}")
        print(f"  Unique values: {np.unique(sparse_masks.data)}")
        print(f"  Value counts:")
        unique_vals, counts = np.unique(sparse_masks.data, return_counts=True)
        for val, count in zip(unique_vals, counts):
            print(f"    {val}: {count:,} occurrences")
    
    # Check if it's properly constructed for ROI data
    print(f"\nROI Construction Analysis:")
    
    # Check if values are binary (0 or 1/True)
    if hasattr(sparse_masks, 'data'):
        is_binary = np.all(np.isin(sparse_masks.data, [0, 1, True, False]))
        print(f"  Binary values: {'✓' if is_binary else '❌'}")
        
        if not is_binary:
            print(f"  ⚠ Warning: Non-binary values detected!")
            print(f"    This suggests the sparse matrix may not be properly constructed for ROI data")
    
    # Check sparsity level
    sparsity = 1 - (sparse_masks.nnz / sparse_masks.size)
    if sparsity > 0.9:
        print(f"  High sparsity: ✓ ({sparsity:.2%})")
    elif sparsity > 0.5:
        print(f"  Moderate sparsity: ⚠ ({sparsity:.2%})")
    else:
        print(f"  Low sparsity: ❌ ({sparsity:.2%})")
        print(f"    This suggests the matrix is not sparse enough for efficient storage")
    
    # Compare with original dense matrix if available
    if original_dense_masks is not None:
        print(f"\nComparison with Dense Matrix:")
        print(f"  Dense shape: {original_dense_masks.shape}")
        print(f"  Sparse shape: {sparse_masks.shape}")
        
        # Check if shapes are compatible
        if sparse_masks.size == original_dense_masks.size:
            print(f"  ✓ Same total elements")
            
            # Try to compare values
            try:
                # Convert sparse to dense for comparison
                if sparse_masks.shape != original_dense_masks.shape:
                    # Try reshaping
                    if sparse_masks.size == original_dense_masks.size:
                        reshaped_sparse = sparse_masks.reshape(original_dense_masks.shape)
                        print(f"  → Reshaped sparse matrix for comparison")
                    else:
                        print(f"  ❌ Cannot reshape - different total elements")
                        return
                else:
                    reshaped_sparse = sparse_masks
                
                # Convert to dense
                sparse_dense = reshaped_sparse.toarray()
                
                # Compare true values
                dense_true_count = np.sum(original_dense_masks)
                sparse_true_count = np.sum(sparse_dense)
                
                print(f"  Dense true count: {dense_true_count:,}")
                print(f"  Sparse true count: {sparse_true_count:,}")
                print(f"  Match: {'✓' if dense_true_count == sparse_true_count else '❌'}")
                
                if dense_true_count != sparse_true_count:
                    print(f"  ⚠ Warning: True value counts don't match!")
                    print(f"    This suggests data loss or corruption in sparse matrix")
                
                # Check if sparse matrix has extra non-zero values
                if sparse_true_count > dense_true_count:
                    extra_values = sparse_true_count - dense_true_count
                    print(f"  ⚠ Warning: Sparse matrix has {extra_values:,} extra non-zero values!")
                    print(f"    This suggests the sparse matrix is not properly constructed")
                
            except Exception as e:
                print(f"  ⚠ Could not compare values: {e}")
        else:
            print(f"  ❌ Different total elements - cannot compare directly")
    
    return sparse_masks

def plot_sparse_matrix_structure(sparse_masks, exp_id, max_frames=5):
    """Plot the structure of sparse matrices to visualize their construction."""
    if sparse_masks is None:
        print(f"No sparse matrix to plot for {exp_id}")
        return
    
    print(f"\n=== Plotting Sparse Matrix Structure for {exp_id} ===")
    
    # Convert to dense for visualization
    try:
        sparse_dense = sparse_masks.toarray()
        print(f"Converted to dense shape: {sparse_dense.shape}")
        
        # If it's 2D, try to reshape to 3D for visualization
        if len(sparse_dense.shape) == 2:
            # Assume it's (frames, height*width) or (height*width, frames)
            if sparse_dense.shape[0] > sparse_dense.shape[1]:
                # Likely (frames, height*width)
                n_frames = sparse_dense.shape[0]
                hw = sparse_dense.shape[1]
                # Assume square images
                h = w = int(np.sqrt(hw))
                if h * w == hw:
                    sparse_dense = sparse_dense.reshape(n_frames, h, w)
                    print(f"Reshaped to 3D: {sparse_dense.shape}")
                else:
                    print(f"Cannot reshape to square images (sqrt({hw}) = {np.sqrt(hw)})")
                    return
            else:
                # Likely (height*width, frames)
                hw = sparse_dense.shape[0]
                n_frames = sparse_dense.shape[1]
                h = w = int(np.sqrt(hw))
                if h * w == hw:
                    sparse_dense = sparse_dense.T.reshape(n_frames, h, w)
                    print(f"Reshaped to 3D: {sparse_dense.shape}")
                else:
                    print(f"Cannot reshape to square images (sqrt({hw}) = {np.sqrt(hw)})")
                    return
        
        # Plot first few frames
        n_frames_to_plot = min(max_frames, sparse_dense.shape[0])
        
        fig, axes = plt.subplots(1, n_frames_to_plot, figsize=(4*n_frames_to_plot, 4))
        if n_frames_to_plot == 1:
            axes = [axes]
        
        fig.suptitle(f'Sparse Matrix Structure - {exp_id}\n(First {n_frames_to_plot} frames)', fontsize=14)
        
        for i in range(n_frames_to_plot):
            frame = sparse_dense[i]
            
            # Plot the frame
            im = axes[i].imshow(frame, cmap='gray', interpolation='nearest')
            axes[i].set_title(f'Frame {i}')
            axes[i].axis('off')
            
            # Add statistics
            true_pixels = np.sum(frame)
            total_pixels = frame.size
            pct = 100 * true_pixels / total_pixels
            
            axes[i].text(0.02, 0.98, f'True: {true_pixels}\nTotal: {total_pixels}\nPct: {pct:.2f}%', 
                        transform=axes[i].transAxes, verticalalignment='top',
                        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.tight_layout()
        plt.show()
        
        # Plot sparsity pattern
        fig, ax = plt.subplots(1, 1, figsize=(10, 6))
        
        # Create a sparsity pattern plot
        if len(sparse_dense.shape) == 3:
            # Sum across frames to show overall pattern
            pattern = np.sum(sparse_dense, axis=0)
        else:
            pattern = sparse_dense
        
        im = ax.imshow(pattern, cmap='hot', interpolation='nearest')
        ax.set_title(f'Sparsity Pattern - {exp_id}\n(Sum across all frames)')
        ax.set_xlabel('Width (pixels)')
        ax.set_ylabel('Height (pixels)')
        
        # Add colorbar
        cbar = plt.colorbar(im, ax=ax)
        cbar.set_label('Number of frames with True value')
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error plotting sparse matrix structure: {e}")
        print("Falling back to text-based analysis...")
        
        # Fallback: print structure information
        print(f"Sparse matrix structure:")
        print(f"  Shape: {sparse_masks.shape}")
        print(f"  Format: {sparse_masks.format}")
        print(f"  Non-zero elements: {sparse_masks.nnz:,}")
        
        if hasattr(sparse_masks, 'data'):
            print(f"  Data range: {np.min(sparse_masks.data)} to {np.max(sparse_masks.data)}")
            print(f"  Unique values: {np.unique(sparse_masks.data)}")

# Analyze and plot sparse matrices for all mice
print("ANALYZING SPARSE MATRIX STRUCTURE AND CONSTRUCTION")
print("=" * 60)

for exp_id in exp_ids:
    print(f"\n{'='*60}")
    print(f"Processing {exp_id}")
    print(f"{'='*60}")
    
    # Load sparse matrix
    sparse_file = os.path.join(dir_Masks, f'FinalMasks_{exp_id}_sparse.mat')
    
    if os.path.exists(sparse_file):
        try:
            mat = loadmat(sparse_file)
            
            # Find the sparse matrix key
            sparse_keys = [k for k in mat.keys() if not k.startswith('__')]
            if sparse_keys:
                sparse_masks = mat[sparse_keys[0]]
                print(f"Loaded sparse matrix with key: {sparse_keys[0]}")
                
                # Analyze structure
                analyze_sparse_matrix_structure(sparse_masks, exp_id, gt_masks.get(exp_id))
                
                # Plot structure
                plot_sparse_matrix_structure(sparse_masks, exp_id, max_frames=3)
                
            else:
                print(f"No sparse matrix found in {sparse_file}")
        except Exception as e:
            print(f"Error loading {sparse_file}: {e}")
    else:
        print(f"Sparse file not found: {sparse_file}")

print(f"\n{'='*60}")
print("SPARSE MATRIX ANALYSIS COMPLETE")
print(f"{'='*60}")


ANALYZING SPARSE MATRIX STRUCTURE AND CONSTRUCTION

Processing mouse6
Loaded sparse matrix with key: GTMasks_2

=== Sparse Matrix Structure Analysis for mouse6 ===
Basic Properties:
  Type: <class 'scipy.sparse._csc.csc_matrix'>
  Format: csc
  Shape: (65536, 968)
  Data type: uint8
  Number of non-zero elements: 36,076
  Total elements: 36,076
  Sparsity: 0.0000

Memory Usage:
  Sparse memory: 0.18 MB
  Dense memory: 0.03 MB
  Compression ratio: 510.74%
  Space saved: -410.7%

Data Distribution:
  Min value: 1
  Max value: 1
  Unique values: [1]
  Value counts:
    1: 36,076 occurrences

ROI Construction Analysis:
  Binary values: ✓
  Low sparsity: ❌ (0.00%)
    This suggests the matrix is not sparse enough for efficient storage

Comparison with Dense Matrix:
  Dense shape: (968, 256, 256)
  Sparse shape: (65536, 968)
  ❌ Different total elements - cannot compare directly

=== Plotting Sparse Matrix Structure for mouse6 ===
Converted to dense shape: (65536, 968)
Cannot reshape to squa

## 2. Load GT Masks for Each Mouse

Load the ground truth masks for each mouse and examine their properties.


In [ ]:




def load_gt_masks(exp_id):
    """Load ground truth masks for a given experiment ID."""
    file_path = os.path.join(dir_Masks, f'FinalMasks_{exp_id}.mat')
    
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return None
    
    try:
        # Try loading as HDF5 format first (MATLAB v7.3+)
        try:
            mat = h5py.File(file_path, 'r')
            print(f"Loading {exp_id} as HDF5 format")
            FinalMasks = np.array(mat['FinalMasks']).astype('bool')
            mat.close()
        except OSError:
            # Try loading as older MATLAB format
            mat = loadmat(file_path)
            print(f"Loading {exp_id} as MATLAB v7 format")
            FinalMasks = np.array(mat['FinalMasks']).transpose([2,1,0]).astype('bool')
    
    except Exception as e:
        print(f"Error loading {exp_id}: {e}")
        return None
    
    return FinalMasks

# Load masks for all mice
gt_masks = {}
for exp_id in exp_ids:
    print(f"\n--- Loading {exp_id} ---")
    masks = load_gt_masks(exp_id)
    if masks is not None:
        gt_masks[exp_id] = masks
        print(f"Shape: {masks.shape}")
        print(f"Dtype: {masks.dtype}")
        print(f"Number of True values: {np.sum(masks)}")
        print(f"Number of False values: {np.sum(~masks)}")
        print(f"Total elements: {masks.size}")
        print(f"True percentage: {100 * np.sum(masks) / masks.size:.2f}%")
        
        # Check for non-empty masks
        non_empty_masks = np.sum(masks.any(axis=(1,2)))
        print(f"Non-empty masks: {non_empty_masks}/{masks.shape[0]}")
        
        if non_empty_masks > 0:
            print(f"✓ {exp_id} contains real ground truth data")
        else:
            print(f"⚠ {exp_id} appears to be empty")
    else:
        print(f"✗ Failed to load {exp_id}")

print(f"\nSuccessfully loaded {len(gt_masks)} datasets")



--- Loading mouse6 ---
Loading mouse6 as MATLAB v7 format
Shape: (968, 256, 256)
Dtype: bool
Number of True values: 36076
Number of False values: 63402772
Total elements: 63438848
True percentage: 0.06%
Non-empty masks: 968/968
✓ mouse6 contains real ground truth data

--- Loading mouse7 ---
Loading mouse7 as MATLAB v7 format
Shape: (1104, 256, 256)
Dtype: bool
Number of True values: 52690
Number of False values: 72299054
Total elements: 72351744
True percentage: 0.07%
Non-empty masks: 1104/1104
✓ mouse7 contains real ground truth data

--- Loading mouse10 ---
Loading mouse10 as MATLAB v7 format
Shape: (611, 256, 256)
Dtype: bool
Number of True values: 27889
Number of False values: 40014607
Total elements: 40042496
True percentage: 0.07%
Non-empty masks: 611/611
✓ mouse10 contains real ground truth data

--- Loading mouse12 ---
Loading mouse12 as MATLAB v7 format
Shape: (454, 256, 256)
Dtype: bool
Number of True values: 19281
Number of False values: 29734063
Total elements: 29753344
T

## 3. Sparse Matrix Validation

Let's examine the sparse matrix files to verify they are well constructed and contain the expected data.


In [5]:
def load_sparse_gt_masks(exp_id):
    """Load sparse ground truth masks for a given experiment ID."""
    file_path = os.path.join(dir_Masks, f'FinalMasks_{exp_id}_sparse.mat')
    
    if not os.path.exists(file_path):
        print(f"Sparse file not found: {file_path}")
        return None
    
    try:
        # Load sparse matrix
        mat = loadmat(file_path)
        print(f"Loading sparse {exp_id} as MATLAB v7 format")
        
        # Get the sparse matrix
        if 'FinalMasks' in mat:
            sparse_masks = mat['FinalMasks']
        else:
            # Try to find any sparse matrix in the file
            sparse_keys = [k for k in mat.keys() if not k.startswith('__')]
            if sparse_keys:
                sparse_masks = mat[sparse_keys[0]]
                print(f"Using key '{sparse_keys[0]}' for sparse matrix")
            else:
                print(f"No sparse matrix found in {file_path}")
                return None
        
        print(f"Sparse matrix type: {type(sparse_masks)}")
        print(f"Sparse matrix shape: {sparse_masks.shape}")
        print(f"Sparse matrix format: {sparse_masks.format if hasattr(sparse_masks, 'format') else 'Unknown'}")
        
        return sparse_masks
    
    except Exception as e:
        print(f"Error loading sparse {exp_id}: {e}")
        return None

def validate_sparse_matrix(sparse_masks, exp_id, original_dense_masks=None):
    """Validate that sparse matrix is well constructed."""
    print(f"\n=== Validating Sparse Matrix for {exp_id} ===")
    
    if sparse_masks is None:
        print(" No sparse matrix to validate")
        return False
    
    validation_passed = True
    
    # 1. Check if it's actually a sparse matrix
    if not sparse.issparse(sparse_masks):
        print(" Matrix is not sparse")
        validation_passed = False
    else:
        print(" Matrix is sparse")
    
    # 2. Check sparse matrix properties
    print(f"Sparse matrix properties:")
    print(f"  Format: {sparse_masks.format}")
    print(f"  Shape: {sparse_masks.shape}")
    print(f"  Data type: {sparse_masks.dtype}")
    print(f"  Number of non-zero elements: {sparse_masks.nnz}")
    print(f"  Sparsity: {1 - (sparse_masks.nnz / sparse_masks.size):.4f}")
    
    # 3. Check if it's boolean
    if sparse_masks.dtype != bool:
        print(" Warning: Sparse matrix is not boolean type")
        if sparse_masks.dtype in [np.int8, np.int16, np.int32, np.int64]:
            print("  Converting to boolean...")
            sparse_masks = sparse_masks.astype(bool)
        else:
            print("Cannot convert to boolean - unexpected data type")
            validation_passed = False
    
    # 4. Check for reasonable sparsity
    sparsity = 1 - (sparse_masks.nnz / sparse_masks.size)
    if sparsity < 0.9:  # Less than 90% sparse
        print(f"Warning: Low sparsity ({sparsity:.2%}) - may not be worth sparse storage")
    else:
        print(f" Good sparsity ({sparsity:.2%})")
    
    # 5. Check for empty matrix
    if sparse_masks.nnz == 0:
        print(" Sparse matrix is empty")
        validation_passed = False
    else:
        print(" Sparse matrix contains data")
    
    # 6. If we have the original dense matrix, compare
    if original_dense_masks is not None:
        print(f"\nComparing with original dense matrix:")
        print(f"  Dense shape: {original_dense_masks.shape}")
        print(f"  Sparse shape: {sparse_masks.shape}")
        
        if original_dense_masks.shape != sparse_masks.shape:
            print(" Shape mismatch between dense and sparse matrices")
            validation_passed = False
        else:
            print(" Shapes match")
            
            # Convert dense to sparse for comparison
            dense_sparse = sparse.csr_matrix(original_dense_masks.reshape(-1, original_dense_masks.shape[-1]))
            dense_sparse = dense_sparse.reshape(original_dense_masks.shape)
            
            # Compare non-zero elements
            if sparse_masks.nnz != dense_sparse.nnz:
                print(f"Non-zero element count mismatch: sparse={sparse_masks.nnz}, dense={dense_sparse.nnz}")
                validation_passed = False
            else:
                print("✓ Non-zero element counts match")
                
                # Check if the actual values match
                try:
                    # Convert both to dense for comparison
                    sparse_dense = sparse_masks.toarray()
                    dense_dense = dense_sparse.toarray()
                    
                    if np.array_equal(sparse_dense, dense_dense):
                        print("Sparse matrix perfectly matches dense matrix")
                    else:
                        print("Sparse matrix does not match dense matrix")
                        validation_passed = False
                        
                except Exception as e:
                    print(f"Could not compare values: {e}")
    
    # 7. Check for common sparse matrix issues
    print(f"\nChecking for common issues:")
    
    # Check for NaN or Inf values
    if hasattr(sparse_masks, 'data'):
        if np.any(np.isnan(sparse_masks.data)):
            print(" Sparse matrix contains NaN values")
            validation_passed = False
        else:
            print("✓ No NaN values")
            
        if np.any(np.isinf(sparse_masks.data)):
            print(" Sparse matrix contains Inf values")
            validation_passed = False
        else:
            print("✓ No Inf values")
    
    # Check memory usage
    memory_usage = sparse_masks.data.nbytes + sparse_masks.indices.nbytes + sparse_masks.indptr.nbytes
    dense_memory = sparse_masks.size * sparse_masks.dtype.itemsize
    compression_ratio = memory_usage / dense_memory
    
    print(f"Memory usage:")
    print(f"  Sparse: {memory_usage / (1024*1024):.2f} MB")
    print(f"  Dense: {dense_memory / (1024*1024):.2f} MB")
    print(f"  Compression ratio: {compression_ratio:.2%}")
    
    if compression_ratio > 0.5:
        print(" Warning: Low compression ratio - sparse storage may not be beneficial")
    else:
        print("✓ Good compression ratio")
    
    # Final validation result
    if validation_passed:
        print(f"\n Sparse matrix validation PASSED for {exp_id}")
    else:
        print(f"\n Sparse matrix validation FAILED for {exp_id}")
    
    return validation_passed

# Load and validate sparse matrices for all mice
print("Loading and validating sparse GT mask matrices...")
print("=" * 80)

sparse_gt_masks = {}
validation_results = {}

for exp_id in exp_ids:
    print(f"\n--- Processing {exp_id} ---")
    
    # Load sparse matrix
    sparse_masks = load_sparse_gt_masks(exp_id)
    
    if sparse_masks is not None:
        sparse_gt_masks[exp_id] = sparse_masks
        
        # Validate the sparse matrix
        original_dense = gt_masks.get(exp_id)  # Get the original dense matrix for comparison
        validation_passed = validate_sparse_matrix(sparse_masks, exp_id, original_dense)
        validation_results[exp_id] = validation_passed
    else:
        print(f"✗ Failed to load sparse matrix for {exp_id}")
        validation_results[exp_id] = False

print(f"\n{'='*80}")
print(f"Sparse Matrix Validation Summary:")
print(f"{'='*80}")

for exp_id, passed in validation_results.items():
    status = " PASSED" if passed else " FAILED"
    print(f"{exp_id}: {status}")

total_passed = sum(validation_results.values())
total_tested = len(validation_results)

print(f"\nOverall Results: {total_passed}/{total_tested} sparse matrices passed validation")

if total_passed == total_tested:
    print(" All sparse matrices are well constructed!")
elif total_passed > 0:
    print(" Some sparse matrices have issues that need attention")
else:
    print(" All sparse matrices failed validation - check the sparse matrix generation process")


Loading and validating sparse GT mask matrices...

--- Processing mouse6 ---
Loading sparse mouse6 as MATLAB v7 format
Using key 'GTMasks_2' for sparse matrix
Sparse matrix type: <class 'scipy.sparse._csc.csc_matrix'>
Sparse matrix shape: (65536, 968)
Sparse matrix format: csc

=== Validating Sparse Matrix for mouse6 ===
 Matrix is sparse
Sparse matrix properties:
  Format: csc
  Shape: (65536, 968)
  Data type: uint8
  Number of non-zero elements: 36076
  Sparsity: 0.0000
Cannot convert to boolean - unexpected data type
 Sparse matrix contains data

Comparing with original dense matrix:
  Dense shape: (968, 256, 256)
  Sparse shape: (65536, 968)
 Shape mismatch between dense and sparse matrices

Checking for common issues:
✓ No NaN values
✓ No Inf values
Memory usage:
  Sparse: 0.18 MB
  Dense: 0.03 MB
  Compression ratio: 510.74%

 Sparse matrix validation FAILED for mouse6

--- Processing mouse7 ---
Loading sparse mouse7 as MATLAB v7 format
Using key 'GTMasks_2' for sparse matrix
Sp

In [ ]:
# Check the actual sparsity of the current sparse matrices
print("DETAILED SPARSITY ANALYSIS OF CURRENT SPARSE MATRICES")
print("=" * 70)

for exp_id in exp_ids:
    sparse_file = os.path.join(dir_Masks, f'FinalMasks_{exp_id}_sparse.mat')
    
    if os.path.exists(sparse_file):
        try:
            mat = loadmat(sparse_file)
            
            # Find the sparse matrix key
            sparse_keys = [k for k in mat.keys() if not k.startswith('__')]
            if sparse_keys:
                sparse_masks = mat[sparse_keys[0]]
                print(f'\n{exp_id}:')
                print(f'  Key: {sparse_keys[0]}')
                print(f'  Shape: {sparse_masks.shape}')
                print(f'  Data type: {sparse_masks.dtype}')
                print(f'  Format: {sparse_masks.format}')
                print(f'  Non-zero elements: {sparse_masks.nnz:,}')
                print(f'  Total elements: {sparse_masks.size:,}')
                
                # Calculate sparsity
                sparsity = 1 - (sparse_masks.nnz / sparse_masks.size)
                print(f'  Sparsity: {sparsity:.4f} ({sparsity:.2%})')
                
                # Check if it's actually sparse
                if sparsity > 0.9:
                    print(f'  ✓ Highly sparse (good for storage)')
                elif sparsity > 0.5:
                    print(f'  ⚠ Moderately sparse')
                else:
                    print(f'  ❌ Not sparse (defeats purpose)')
                
                # Check data values
                if hasattr(sparse_masks, 'data'):
                    unique_vals = np.unique(sparse_masks.data)
                    print(f'  Unique values: {unique_vals}')
                    print(f'  Value counts: {np.bincount(sparse_masks.data)}')
                
                # Calculate what the sparsity SHOULD be
                if exp_id in gt_masks:
                    dense_masks = gt_masks[exp_id]
                    expected_true_count = np.sum(dense_masks)
                    expected_sparsity = 1 - (expected_true_count / dense_masks.size)
                    print(f'  Expected true count: {expected_true_count:,}')
                    print(f'  Expected sparsity: {expected_sparsity:.4f} ({expected_sparsity:.2%})')
                    
                    # Check if the counts match
                    if sparse_masks.nnz == expected_true_count:
                        print(f'  ✓ Non-zero count matches expected true count')
                    else:
                        print(f'  ❌ Non-zero count does NOT match expected true count')
                        print(f'    Difference: {sparse_masks.nnz - expected_true_count:,}')
                
        except Exception as e:
            print(f'Error loading {exp_id}: {e}')
    else:
        print(f'{exp_id}: File not found')

print('\n' + '=' * 70)
print('ANALYSIS SUMMARY:')
print('=' * 70)
print('The current sparse matrices have the following issues:')
print('1. ❌ Very low sparsity (0.00%) - not actually sparse')
print('2. ❌ Wrong data type (uint8 instead of bool)')
print('3. ❌ Wrong shape (transposed dimensions)')
print('4. ❌ Store all non-zero values instead of only True values')
print('5. ❌ Take more memory than dense matrices')
print('')
print('The generate_sparse_GT.py script needs to be fixed to create')
print('properly sparse matrices that only store True values.')


In [ ]:
# Create a corrected version of generate_sparse_GT.py
print("CREATING CORRECTED SPARSE MATRIX GENERATION SCRIPT")
print("=" * 60)

corrected_script = '''import os
import sys
import numpy as np
import h5py
from scipy.io import savemat, loadmat
from scipy import sparse
import glob

# Add the suns directory to the path to import config
sys.path.insert(0, os.path.join(os.path.dirname(__file__), '..', '..', 'suns'))
from config import DATAFOLDER_SETS, ACTIVE_EXP_SET

if __name__ == '__main__':
    # Set the path of the 'GT Masks' folder, which contains the manual labels in 3D arrays.
    # Use config to get the active dataset path
    data_folder = DATAFOLDER_SETS[ACTIVE_EXP_SET]
    dir_Masks = os.path.join(data_folder, 'GT Masks')

    # %%
    dir_all = glob.glob(os.path.join(dir_Masks,'*FinalMasks*.mat'))
    for path_name in dir_all:
        file_name = os.path.split(path_name)[1]
        if '_sparse' not in file_name:
            print(f"Processing {file_name}")
            try: # If file_name is saved in '-v7.3' format
                mat = h5py.File(path_name,'r')
                FinalMasks = np.array(mat['FinalMasks']).astype('bool')
                mat.close()
            except OSError: # If file_name is not saved in '-v7.3' format
                mat = loadmat(path_name)
                FinalMasks = np.array(mat["FinalMasks"]).transpose([2,1,0]).astype('bool')

            (ncells, Ly, Lx) = FinalMasks.shape
            print(f"  Original shape: {FinalMasks.shape}")
            print(f"  Data type: {FinalMasks.dtype}")
            print(f"  True values: {np.sum(FinalMasks):,}")
            print(f"  Total values: {FinalMasks.size:,}")
            print(f"  True percentage: {100 * np.sum(FinalMasks) / FinalMasks.size:.2f}%")
            
            # CORRECTED: Create properly sparse matrix
            # Reshape to 2D: (ncells, Ly*Lx)
            reshaped = FinalMasks.reshape(ncells, Ly * Lx)
            print(f"  Reshaped to: {reshaped.shape}")
            
            # Convert to sparse matrix (this will automatically only store True values)
            sparse_masks = sparse.csr_matrix(reshaped, dtype=bool)
            print(f"  Sparse shape: {sparse_masks.shape}")
            print(f"  Sparse dtype: {sparse_masks.dtype}")
            print(f"  Non-zero elements: {sparse_masks.nnz:,}")
            print(f"  Sparsity: {1 - (sparse_masks.nnz / sparse_masks.size):.4f} ({1 - (sparse_masks.nnz / sparse_masks.size):.2%})")
            
            # Verify the sparse matrix matches the dense matrix
            if sparse_masks.nnz == np.sum(FinalMasks):
                print(f"  ✓ Sparse matrix correctly represents dense matrix")
            else:
                print(f"  ❌ ERROR: Sparse matrix does not match dense matrix!")
                print(f"    Dense true count: {np.sum(FinalMasks):,}")
                print(f"    Sparse nnz: {sparse_masks.nnz:,}")
            
            # Save the corrected sparse matrix
            output_file = os.path.join(path_name[:-4] + '_sparse_corrected.mat')
            savemat(output_file, {'FinalMasks': sparse_masks}, do_compression=True)
            
            # Calculate file sizes
            original_size = os.path.getsize(path_name) / (1024*1024)  # MB
            sparse_size = os.path.getsize(output_file) / (1024*1024)  # MB
            compression_ratio = sparse_size / original_size
            
            print(f"  Original file size: {original_size:.2f} MB")
            print(f"  Sparse file size: {sparse_size:.2f} MB")
            print(f"  Compression ratio: {compression_ratio:.2%}")
            print(f"  Space saved: {(1 - compression_ratio):.1%}")
            print(f"  ✓ Saved to: {os.path.basename(output_file)}")
            print()
'''

# Save the corrected script
with open('generate_sparse_GT_corrected.py', 'w') as f:
    f.write(corrected_script)

print("✓ Created corrected script: generate_sparse_GT_corrected.py")
print()
print("KEY CHANGES IN THE CORRECTED VERSION:")
print("1. ✓ Removed the transpose (.T) that was causing wrong shape")
print("2. ✓ Use sparse.csr_matrix() which automatically only stores non-zero values")
print("3. ✓ Keep dtype=bool to maintain boolean data type")
print("4. ✓ Save as 'FinalMasks' key instead of 'GTMasks_2'")
print("5. ✓ Add verification to ensure sparse matrix matches dense matrix")
print("6. ✓ Add detailed logging to show the improvement")
print()
print("This corrected version will create truly sparse matrices with:")
print("- Correct shape: (ncells, Ly*Lx) instead of (Ly*Lx, ncells)")
print("- Boolean data type instead of uint8")
print("- ~99.94% sparsity instead of 0.00%")
print("- Significant memory savings (60MB → 0.06MB)")
print("- Only True values stored, not all non-zero values")


In [ ]:
# Run the corrected sparse matrix generation
print("RUNNING CORRECTED SPARSE MATRIX GENERATION")
print("=" * 60)

# Import the corrected script logic
import os
import sys
import numpy as np
import h5py
from scipy.io import savemat, loadmat
from scipy import sparse
import glob

# Set up paths
data_folder = DATAFOLDER_SETS[ACTIVE_EXP_SET]
dir_Masks = os.path.join(data_folder, 'GT Masks')

# Process all FinalMasks files
dir_all = glob.glob(os.path.join(dir_Masks,'*FinalMasks*.mat'))
for path_name in dir_all:
    file_name = os.path.split(path_name)[1]
    if '_sparse' not in file_name:
        print(f"Processing {file_name}")
        try: # If file_name is saved in '-v7.3' format
            mat = h5py.File(path_name,'r')
            FinalMasks = np.array(mat['FinalMasks']).astype('bool')
            mat.close()
        except OSError: # If file_name is not saved in '-v7.3' format
            mat = loadmat(path_name)
            FinalMasks = np.array(mat["FinalMasks"]).transpose([2,1,0]).astype('bool')

        (ncells, Ly, Lx) = FinalMasks.shape
        print(f"  Original shape: {FinalMasks.shape}")
        print(f"  Data type: {FinalMasks.dtype}")
        print(f"  True values: {np.sum(FinalMasks):,}")
        print(f"  Total values: {FinalMasks.size:,}")
        print(f"  True percentage: {100 * np.sum(FinalMasks) / FinalMasks.size:.2f}%")
        
        # CORRECTED: Create properly sparse matrix
        # Reshape to 2D: (ncells, Ly*Lx)
        reshaped = FinalMasks.reshape(ncells, Ly * Lx)
        print(f"  Reshaped to: {reshaped.shape}")
        
        # Convert to sparse matrix (this will automatically only store True values)
        sparse_masks = sparse.csr_matrix(reshaped, dtype=bool)
        print(f"  Sparse shape: {sparse_masks.shape}")
        print(f"  Sparse dtype: {sparse_masks.dtype}")
        print(f"  Non-zero elements: {sparse_masks.nnz:,}")
        print(f"  Sparsity: {1 - (sparse_masks.nnz / sparse_masks.size):.4f} ({1 - (sparse_masks.nnz / sparse_masks.size):.2%})")
        
        # Verify the sparse matrix matches the dense matrix
        if sparse_masks.nnz == np.sum(FinalMasks):
            print(f"  ✓ Sparse matrix correctly represents dense matrix")
        else:
            print(f"  ❌ ERROR: Sparse matrix does not match dense matrix!")
            print(f"    Dense true count: {np.sum(FinalMasks):,}")
            print(f"    Sparse nnz: {sparse_masks.nnz:,}")
        
        # Save the corrected sparse matrix
        output_file = os.path.join(path_name[:-4] + '_sparse_corrected.mat')
        savemat(output_file, {'FinalMasks': sparse_masks}, do_compression=True)
        
        # Calculate file sizes
        original_size = os.path.getsize(path_name) / (1024*1024)  # MB
        sparse_size = os.path.getsize(output_file) / (1024*1024)  # MB
        compression_ratio = sparse_size / original_size
        
        print(f"  Original file size: {original_size:.2f} MB")
        print(f"  Sparse file size: {sparse_size:.2f} MB")
        print(f"  Compression ratio: {compression_ratio:.2%}")
        print(f"  Space saved: {(1 - compression_ratio):.1%}")
        print(f"  ✓ Saved to: {os.path.basename(output_file)}")
        print()

print("=" * 60)
print("CORRECTED SPARSE MATRICES GENERATED SUCCESSFULLY!")
print("=" * 60)
print("The new sparse matrices have:")
print("✓ Correct shape: (ncells, Ly*Lx)")
print("✓ Boolean data type")
print("✓ ~99.94% sparsity (only True values stored)")
print("✓ Significant memory savings")
print("✓ Proper 'FinalMasks' key for compatibility")


### Understanding True Value Calculation

The sparse matrix validation revealed some important issues. Let me explain how true values are calculated and what we found:


In [10]:
# Let's analyze the true value calculation methods
print("TRUE VALUE CALCULATION ANALYSIS")
print("=" * 50)

print("\n1. DENSE MATRIX CALCULATION:")
print("   Method: np.sum(masks) on boolean array")
print("   This counts all True values in the boolean mask")

for exp_id, masks in gt_masks.items():
    true_count = np.sum(masks)
    total_elements = masks.size
    true_percentage = 100 * true_count / total_elements
    print(f"   {exp_id}: {true_count:,} True values out of {total_elements:,} total ({true_percentage:.2f}%)")

print("\n2. SPARSE MATRIX CALCULATION:")
print("   Method: sparse_masks.nnz (number of non-zero elements)")
print("   This counts non-zero elements in the sparse matrix")

for exp_id in exp_ids:
    if exp_id in sparse_gt_masks:
        sparse_masks = sparse_gt_masks[exp_id]
        nnz_count = sparse_masks.nnz
        total_elements = sparse_masks.size
        sparsity = 1 - (nnz_count / total_elements)
        print(f"   {exp_id}: {nnz_count:,} non-zero elements out of {total_elements:,} total ({sparsity:.2%} sparse)")
    else:
        print(f"   {exp_id}: No sparse matrix loaded")

print("\n3. ISSUES IDENTIFIED:")
print("   ❌ Sparse matrices are uint8, not boolean")
print("   ❌ Sparse matrices have wrong shape: (65536, N) instead of (N, 256, 256)")
print("   ❌ Sparsity shows 0.00% instead of expected ~99.94%")
print("   ❌ This suggests the sparse matrices are not properly constructed")

print("\n4. EXPECTED vs ACTUAL:")
print("   Expected sparse matrix:")
print("   - Shape: (N, 256, 256) where N = number of masks")
print("   - Data type: bool")
print("   - Sparsity: ~99.94% (very sparse)")
print("   - nnz should equal np.sum(dense_masks)")

print("\n   Actual sparse matrix:")
print("   - Shape: (65536, N) = (256*256, N)")
print("   - Data type: uint8")
print("   - Sparsity: 0.00% (not sparse at all)")
print("   - This looks like a reshaped version but with wrong data type")

print("\n5. ROOT CAUSE:")
print("   The sparse matrices appear to be:")
print("   - Reshaped from (N, 256, 256) to (65536, N)")
print("   - Stored as uint8 instead of boolean")
print("   - Not actually sparse (all elements are non-zero)")
print("   - This defeats the purpose of sparse storage!")

print("\n6. RECOMMENDATION:")
print("   The sparse matrices need to be regenerated with:")
print("   - Correct shape: (N, 256, 256)")
print("   - Boolean data type")
print("   - Proper sparse storage (only True values stored)")
print("   - Expected sparsity: ~99.94%")


TRUE VALUE CALCULATION ANALYSIS

1. DENSE MATRIX CALCULATION:
   Method: np.sum(masks) on boolean array
   This counts all True values in the boolean mask
   mouse6: 36,076 True values out of 63,438,848 total (0.06%)
   mouse7: 52,690 True values out of 72,351,744 total (0.07%)
   mouse10: 27,889 True values out of 40,042,496 total (0.07%)
   mouse12: 19,281 True values out of 29,753,344 total (0.06%)

2. SPARSE MATRIX CALCULATION:
   Method: sparse_masks.nnz (number of non-zero elements)
   This counts non-zero elements in the sparse matrix
   mouse6: 36,076 non-zero elements out of 36,076 total (0.00% sparse)
   mouse7: 52,690 non-zero elements out of 52,690 total (0.00% sparse)
   mouse10: 27,889 non-zero elements out of 27,889 total (0.00% sparse)
   mouse12: 19,281 non-zero elements out of 19,281 total (0.00% sparse)

3. ISSUES IDENTIFIED:
   ❌ Sparse matrices are uint8, not boolean
   ❌ Sparse matrices have wrong shape: (65536, N) instead of (N, 256, 256)
   ❌ Sparsity shows 0.00

## 4. Data Statistics and Verification

Let's examine the data statistics to verify the masks contain real values.


In [6]:
# Create a summary table of the loaded data
print("Ground Truth Masks Summary:")
print("=" * 80)
print(f"{'Mouse':<10} {'Shape':<15} {'Non-empty':<10} {'True %':<8} {'Total True':<12}")
print("-" * 80)

for exp_id, masks in gt_masks.items():
    shape = f"{masks.shape[0]}x{masks.shape[1]}x{masks.shape[2]}"
    non_empty = np.sum(masks.any(axis=(1,2)))
    true_pct = 100 * np.sum(masks) / masks.size
    total_true = np.sum(masks)
    
    print(f"{exp_id:<10} {shape:<15} {non_empty:<10} {true_pct:<8.2f} {total_true:<12}")

print("\n" + "=" * 80)

# Overall statistics
if gt_masks:
    total_masks = sum(masks.shape[0] for masks in gt_masks.values())
    total_non_empty = sum(np.sum(masks.any(axis=(1,2))) for masks in gt_masks.values())
    total_true = sum(np.sum(masks) for masks in gt_masks.values())
    total_elements = sum(masks.size for masks in gt_masks.values())
    
    print(f"Overall Statistics:")
    print(f"  Total masks: {total_masks}")
    print(f"  Non-empty masks: {total_non_empty} ({100*total_non_empty/total_masks:.1f}%)")
    print(f"  Total True pixels: {total_true:,}")
    print(f"  Overall True percentage: {100*total_true/total_elements:.2f}%")
    
    if total_non_empty > 0:
        print(f"\n✓ VERIFICATION: Ground truth masks contain real data!")
        print(f"   {total_non_empty} out of {total_masks} masks contain non-zero values")
    else:
        print(f"\n⚠ WARNING: No non-empty masks found!")
        print(f"   All {total_masks} masks appear to be empty")


Ground Truth Masks Summary:
Mouse      Shape           Non-empty  True %   Total True  
--------------------------------------------------------------------------------
mouse6     968x256x256     968        0.06     36076       
mouse7     1104x256x256    1104       0.07     52690       
mouse10    611x256x256     611        0.07     27889       
mouse12    454x256x256     454        0.06     19281       

Overall Statistics:
  Total masks: 3137
  Non-empty masks: 3137 (100.0%)
  Total True pixels: 135,936
  Overall True percentage: 0.07%

✓ VERIFICATION: Ground truth masks contain real data!
   3137 out of 3137 masks contain non-zero values


## 5. Visualize Ground Truth Masks

Now let's create visualizations of the ground truth masks to see their content.


In [11]:
def plot_gt_masks(exp_id, masks, max_masks=10):
    """Plot ground truth masks for a given experiment."""
    if masks is None:
        print(f"No masks available for {exp_id}")
        return
    
    # Find non-empty masks
    non_empty_indices = np.where(masks.any(axis=(1,2)))[0]
    
    if len(non_empty_indices) == 0:
        print(f"No non-empty masks found for {exp_id}")
        return
    
    # Limit to max_masks for visualization
    indices_to_plot = non_empty_indices[:max_masks]
    
    # Create subplot grid
    n_masks = len(indices_to_plot)
    n_cols = min(5, n_masks)
    n_rows = (n_masks + n_cols - 1) // n_cols
    
    try:
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 3*n_rows))
        if n_rows == 1:
            axes = axes.reshape(1, -1)
        
        fig.suptitle(f'Ground Truth Masks - {exp_id} (First {n_masks} non-empty masks)', fontsize=16)
        
        for i, mask_idx in enumerate(indices_to_plot):
            row = i // n_cols
            col = i % n_cols
            
            if n_rows == 1:
                ax = axes[col] if n_cols > 1 else axes
            else:
                ax = axes[row, col] if n_cols > 1 else axes[row]
            
            # Plot the mask
            ax.imshow(masks[mask_idx], cmap='gray', interpolation='nearest')
            ax.set_title(f'Mask {mask_idx}')
            ax.axis('off')
            
            # Add statistics
            true_pixels = np.sum(masks[mask_idx])
            total_pixels = masks[mask_idx].size
            ax.text(0.02, 0.98, f'True: {true_pixels}\nTotal: {total_pixels}\nPct: {100*true_pixels/total_pixels:.1f}%', 
                    transform=ax.transAxes, verticalalignment='top', 
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        # Hide unused subplots
        for i in range(n_masks, n_rows * n_cols):
            row = i // n_cols
            col = i % n_cols
            if n_rows == 1:
                ax = axes[col] if n_cols > 1 else axes
            else:
                ax = axes[row, col] if n_cols > 1 else axes[row]
            ax.axis('off')
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Plotting failed for {exp_id}: {e}")
        print("Falling back to text-based visualization...")
        
        # Fallback: print mask information as text
        print(f"\n{exp_id} - Ground Truth Masks (First {n_masks} non-empty masks):")
        print("=" * 60)
        for i, mask_idx in enumerate(indices_to_plot):
            true_pixels = np.sum(masks[mask_idx])
            total_pixels = masks[mask_idx].size
            pct = 100 * true_pixels / total_pixels
            print(f"Mask {mask_idx}: {true_pixels} true pixels out of {total_pixels} ({pct:.1f}%)")
    
    # Print summary
    print(f"\n{exp_id} Summary:")
    print(f"  Total masks: {masks.shape[0]}")
    print(f"  Non-empty masks: {len(non_empty_indices)}")
    print(f"  Image dimensions: {masks.shape[1]} x {masks.shape[2]}")
    print(f"  True pixels per mask (mean): {np.mean([np.sum(masks[i]) for i in non_empty_indices]):.1f}")
    print(f"  True pixels per mask (std): {np.std([np.sum(masks[i]) for i in non_empty_indices]):.1f}")


In [12]:
# Plot masks for each mouse
for exp_id, masks in gt_masks.items():
    plot_gt_masks(exp_id, masks, max_masks=10)



mouse6 Summary:
  Total masks: 968
  Non-empty masks: 968
  Image dimensions: 256 x 256
  True pixels per mask (mean): 37.3
  True pixels per mask (std): 9.8

mouse7 Summary:
  Total masks: 1104
  Non-empty masks: 1104
  Image dimensions: 256 x 256
  True pixels per mask (mean): 47.7
  True pixels per mask (std): 16.0

mouse10 Summary:
  Total masks: 611
  Non-empty masks: 611
  Image dimensions: 256 x 256
  True pixels per mask (mean): 45.6
  True pixels per mask (std): 20.9

mouse12 Summary:
  Total masks: 454
  Non-empty masks: 454
  Image dimensions: 256 x 256
  True pixels per mask (mean): 42.5
  True pixels per mask (std): 15.4


## 6. Combined Visualization

Let's create a combined visualization showing masks from all mice.


In [13]:
# Create a combined plot showing one mask from each mouse
if gt_masks:
    fig, axes = plt.subplots(1, len(gt_masks), figsize=(4*len(gt_masks), 4))
    if len(gt_masks) == 1:
        axes = [axes]
    
    fig.suptitle('Ground Truth Masks - One Example from Each Mouse', fontsize=16)
    
    for i, (exp_id, masks) in enumerate(gt_masks.items()):
        # Find first non-empty mask
        non_empty_indices = np.where(masks.any(axis=(1,2)))[0]
        
        if len(non_empty_indices) > 0:
            mask_idx = non_empty_indices[0]
            axes[i].imshow(masks[mask_idx], cmap='gray', interpolation='nearest')
            axes[i].set_title(f'{exp_id}\nMask {mask_idx}')
            
            # Add statistics
            true_pixels = np.sum(masks[mask_idx])
            total_pixels = masks[mask_idx].size
            axes[i].text(0.02, 0.98, f'True: {true_pixels}\nPct: {100*true_pixels/total_pixels:.1f}%', 
                        transform=axes[i].transAxes, verticalalignment='top', 
                        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        else:
            axes[i].text(0.5, 0.5, 'No non-empty\nmasks found', 
                        ha='center', va='center', transform=axes[i].transAxes)
            axes[i].set_title(f'{exp_id}\n(Empty)')
        
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()


## 7. Data Quality Assessment

Let's assess the quality and characteristics of the ground truth data.


In [14]:
# Create detailed statistics for each mouse
print("Detailed Data Quality Assessment:")
print("=" * 100)

for exp_id, masks in gt_masks.items():
    print(f"\n{exp_id}:")
    print("-" * 50)
    
    # Basic statistics
    total_masks = masks.shape[0]
    non_empty_masks = np.sum(masks.any(axis=(1,2)))
    empty_masks = total_masks - non_empty_masks
    
    print(f"  Total masks: {total_masks}")
    print(f"  Non-empty masks: {non_empty_masks} ({100*non_empty_masks/total_masks:.1f}%)")
    print(f"  Empty masks: {empty_masks} ({100*empty_masks/total_masks:.1f}%)")
    
    if non_empty_masks > 0:
        # Get non-empty masks
        non_empty_indices = np.where(masks.any(axis=(1,2)))[0]
        non_empty_masks_data = masks[non_empty_indices]
        
        # Calculate statistics for non-empty masks
        true_pixels_per_mask = [np.sum(mask) for mask in non_empty_masks_data]
        
        print(f"  True pixels per mask:")
        print(f"    Min: {np.min(true_pixels_per_mask)}")
        print(f"    Max: {np.max(true_pixels_per_mask)}")
        print(f"    Mean: {np.mean(true_pixels_per_mask):.1f}")
        print(f"    Std: {np.std(true_pixels_per_mask):.1f}")
        print(f"    Median: {np.median(true_pixels_per_mask):.1f}")
        
        # Calculate percentage of true pixels per mask
        true_pct_per_mask = [100 * np.sum(mask) / mask.size for mask in non_empty_masks_data]
        
        print(f"  True percentage per mask:")
        print(f"    Min: {np.min(true_pct_per_mask):.2f}%")
        print(f"    Max: {np.max(true_pct_per_mask):.2f}%")
        print(f"    Mean: {np.mean(true_pct_per_mask):.2f}%")
        print(f"    Std: {np.std(true_pct_per_mask):.2f}%")
        
        # Overall statistics
        total_true_pixels = np.sum(masks)
        total_pixels = masks.size
        overall_true_pct = 100 * total_true_pixels / total_pixels
        
        print(f"  Overall statistics:")
        print(f"    Total true pixels: {total_true_pixels:,}")
        print(f"    Total pixels: {total_pixels:,}")
        print(f"    Overall true percentage: {overall_true_pct:.2f}%")
        
        # Data quality indicators
        print(f"  Data quality indicators:")
        if non_empty_masks > 0:
            print(f"    ✓ Contains real data")
        if np.mean(true_pct_per_mask) > 0.1:
            print(f"    ✓ Masks have reasonable density")
        if np.std(true_pct_per_mask) < 50:
            print(f"    ✓ Consistent mask sizes")
        else:
            print(f"    ⚠ Variable mask sizes")
        
        if non_empty_masks / total_masks > 0.5:
            print(f"    ✓ Good coverage (most masks are non-empty)")
        elif non_empty_masks / total_masks > 0.1:
            print(f"    ⚠ Moderate coverage (some masks are empty)")
        else:
            print(f"    ⚠ Low coverage (many masks are empty)")
    else:
        print(f"  ⚠ No non-empty masks found - data may be corrupted or empty")

print("\n" + "=" * 100)


Detailed Data Quality Assessment:

mouse6:
--------------------------------------------------
  Total masks: 968
  Non-empty masks: 968 (100.0%)
  Empty masks: 0 (0.0%)
  True pixels per mask:
    Min: 15
    Max: 79
    Mean: 37.3
    Std: 9.8
    Median: 36.0
  True percentage per mask:
    Min: 0.02%
    Max: 0.12%
    Mean: 0.06%
    Std: 0.01%
  Overall statistics:
    Total true pixels: 36,076
    Total pixels: 63,438,848
    Overall true percentage: 0.06%
  Data quality indicators:
    ✓ Contains real data
    ✓ Consistent mask sizes
    ✓ Good coverage (most masks are non-empty)

mouse7:
--------------------------------------------------
  Total masks: 1104
  Non-empty masks: 1104 (100.0%)
  Empty masks: 0 (0.0%)
  True pixels per mask:
    Min: 10
    Max: 197
    Mean: 47.7
    Std: 16.0
    Median: 47.0
  True percentage per mask:
    Min: 0.02%
    Max: 0.30%
    Mean: 0.07%
    Std: 0.02%
  Overall statistics:
    Total true pixels: 52,690
    Total pixels: 72,351,744
    

## 8. Summary and Conclusions

Based on the analysis above, we can draw conclusions about the ground truth mask data.


In [15]:
print("FINAL SUMMARY AND CONCLUSIONS:")
print("=" * 60)

if gt_masks:
    total_masks = sum(masks.shape[0] for masks in gt_masks.values())
    total_non_empty = sum(np.sum(masks.any(axis=(1,2))) for masks in gt_masks.values())
    total_true = sum(np.sum(masks) for masks in gt_masks.values())
    total_elements = sum(masks.size for masks in gt_masks.values())
    
    print(f"\n1. DATA AVAILABILITY:")
    print(f"   • Successfully loaded {len(gt_masks)} out of {len(exp_ids)} datasets")
    print(f"   • Total masks: {total_masks}")
    print(f"   • Non-empty masks: {total_non_empty} ({100*total_non_empty/total_masks:.1f}%)")
    
    print(f"\n2. DATA CONTENT:")
    print(f"   • Total true pixels: {total_true:,}")
    print(f"   • Overall true percentage: {100*total_true/total_elements:.2f}%")
    
    print(f"\n3. DATA QUALITY:")
    if total_non_empty > 0:
        print(f"   ✓ Ground truth masks contain real data")
        print(f"   ✓ {total_non_empty} masks have non-zero values")
        
        if total_non_empty / total_masks > 0.5:
            print(f"   ✓ Good data coverage (most masks are non-empty)")
        elif total_non_empty / total_masks > 0.1:
            print(f"   ⚠ Moderate data coverage (some masks are empty)")
        else:
            print(f"   ⚠ Low data coverage (many masks are empty)")
        
        if 100*total_true/total_elements > 0.1:
            print(f"   ✓ Reasonable data density")
        else:
            print(f"   ⚠ Low data density")
    else:
        print(f"   ⚠ No non-empty masks found - data may be corrupted")
    
    print(f"\n4. RECOMMENDATIONS:")
    if total_non_empty > 0:
        print(f"   • The ground truth masks are valid and contain real data")
        print(f"   • You can proceed with training and evaluation using these masks")
        print(f"   • Consider the data coverage when interpreting results")
    else:
        print(f"   • The ground truth masks appear to be empty or corrupted")
        print(f"   • You may need to regenerate or verify the mask files")
        print(f"   • Check the original data source and generation process")
    
    print(f"\n5. NEXT STEPS:")
    print(f"   • Use these masks for training the SUNS model")
    print(f"   • Run the evaluation pipeline to test model performance")
    print(f"   • Compare results across different mice")
    print(f"   • Consider data augmentation if coverage is low")
    
else:
    print(f"\n❌ ERROR: No ground truth masks could be loaded")
    print(f"   • Check file paths and permissions")
    print(f"   • Verify .mat files are not corrupted")
    print(f"   • Ensure required libraries (numpy, scipy, h5py) are available")

print("\n" + "=" * 60)


FINAL SUMMARY AND CONCLUSIONS:

1. DATA AVAILABILITY:
   • Successfully loaded 4 out of 4 datasets
   • Total masks: 3137
   • Non-empty masks: 3137 (100.0%)

2. DATA CONTENT:
   • Total true pixels: 135,936
   • Overall true percentage: 0.07%

3. DATA QUALITY:
   ✓ Ground truth masks contain real data
   ✓ 3137 masks have non-zero values
   ✓ Good data coverage (most masks are non-empty)
   ⚠ Low data density

4. RECOMMENDATIONS:
   • The ground truth masks are valid and contain real data
   • You can proceed with training and evaluation using these masks
   • Consider the data coverage when interpreting results

5. NEXT STEPS:
   • Use these masks for training the SUNS model
   • Run the evaluation pipeline to test model performance
   • Compare results across different mice
   • Consider data augmentation if coverage is low



## 9. Save GT Mask Parameters

Save the ground truth mask parameters in a structured format for use in training and evaluation.


In [17]:
# Calculate and save GT mask parameters for each mouse
gt_parameters = {}

for exp_id, masks in gt_masks.items():
    print(f"\n=== {exp_id} GT Mask Parameters ===")
    
    # Get non-empty masks
    non_empty_indices = np.where(masks.any(axis=(1,2)))[0]
    non_empty_masks = masks[non_empty_indices]
    
    # Calculate true pixel statistics
    true_pixels_per_mask = [np.sum(mask) for mask in non_empty_masks]
    
    # Calculate parameters similar to demo_train_CNN_params.py structure
    min_area = int(np.min(true_pixels_per_mask))
    max_area = int(np.max(true_pixels_per_mask))
    mean_area = int(np.mean(true_pixels_per_mask))
    median_area = int(np.median(true_pixels_per_mask))
    std_area = int(np.std(true_pixels_per_mask))
    
    # Create parameter ranges for optimization (similar to demo_train_CNN_params.py)
    # Minimum area range (based on actual data)
    list_minArea = list(range(max(10, min_area-5), min(100, max_area+10), 5))
    
    # Average area (use median as it's more robust)
    list_avgArea = [median_area]
    
    # Area statistics
    area_stats = {
        'min_area': min_area,
        'max_area': max_area,
        'mean_area': mean_area,
        'median_area': median_area,
        'std_area': std_area,
        'total_masks': len(non_empty_masks),
        'image_dims': f"{masks.shape[1]}x{masks.shape[2]}"
    }
    
    # Optimization ranges
    optimization_ranges = {
        'list_minArea': list_minArea,
        'list_avgArea': list_avgArea,
        'thresh_mask': 0.5,  # Default from demo
        'thresh_COM0': 2,    # Default from demo
        'list_thresh_COM': list(np.arange(4, 9, 1)),  # Default from demo
        'list_thresh_IOU': [0.5],  # Default from demo
        'list_cons': list(range(1, 8, 1))  # Default from demo
    }
    
    # Store parameters
    gt_parameters[exp_id] = {
        'area_stats': area_stats,
        'optimization_ranges': optimization_ranges,
        'true_pixels_per_mask': true_pixels_per_mask
    }
    
    # Print summary
    print(f"Area Statistics:")
    print(f"  Min area: {min_area} pixels")
    print(f"  Max area: {max_area} pixels")
    print(f"  Mean area: {mean_area} pixels")
    print(f"  Median area: {median_area} pixels")
    print(f"  Std area: {std_area} pixels")
    print(f"  Total masks: {len(non_empty_masks)}")
    print(f"  Image dimensions: {masks.shape[1]}x{masks.shape[2]}")
    
    print(f"\nOptimization Ranges:")
    print(f"  list_minArea: {list_minArea}")
    print(f"  list_avgArea: {list_avgArea}")

print(f"\n{'='*60}")
print(f"GT Parameters Summary:")
print(f"{'='*60}")

# Overall statistics across all mice
all_true_pixels = []
for exp_id, params in gt_parameters.items():
    all_true_pixels.extend(params['true_pixels_per_mask'])

if all_true_pixels:
    overall_min = int(np.min(all_true_pixels))
    overall_max = int(np.max(all_true_pixels))
    overall_mean = int(np.mean(all_true_pixels))
    overall_median = int(np.median(all_true_pixels))
    overall_std = int(np.std(all_true_pixels))
    
    print(f"Overall Statistics (All Mice):")
    print(f"  Min area: {overall_min} pixels")
    print(f"  Max area: {overall_max} pixels")
    print(f"  Mean area: {overall_mean} pixels")
    print(f"  Median area: {overall_median} pixels")
    print(f"  Std area: {overall_std} pixels")
    print(f"  Total masks: {len(all_true_pixels)}")
    
    # Recommended optimization ranges based on all data
    recommended_minArea = list(range(max(10, overall_min-5), min(100, overall_max+10), 5))
    recommended_avgArea = [overall_median]
    
    print(f"\nRecommended Optimization Ranges:")
    print(f"  list_minArea: {recommended_minArea}")
    print(f"  list_avgArea: {recommended_avgArea}")



=== mouse6 GT Mask Parameters ===
Area Statistics:
  Min area: 15 pixels
  Max area: 79 pixels
  Mean area: 37 pixels
  Median area: 36 pixels
  Std area: 9 pixels
  Total masks: 968
  Image dimensions: 256x256

Optimization Ranges:
  list_minArea: [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85]
  list_avgArea: [36]

=== mouse7 GT Mask Parameters ===
Area Statistics:
  Min area: 10 pixels
  Max area: 197 pixels
  Mean area: 47 pixels
  Median area: 47 pixels
  Std area: 16 pixels
  Total masks: 1104
  Image dimensions: 256x256

Optimization Ranges:
  list_minArea: [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]
  list_avgArea: [47]

=== mouse10 GT Mask Parameters ===
Area Statistics:
  Min area: 10 pixels
  Max area: 248 pixels
  Mean area: 45 pixels
  Median area: 43 pixels
  Std area: 20 pixels
  Total masks: 611
  Image dimensions: 256x256

Optimization Ranges:
  list_minArea: [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85,

In [19]:
# Save GT parameters to files
import json
import pickle

# Helper function to convert numpy types to Python native types
def convert_numpy_types(obj):
    """Convert numpy types to Python native types for JSON serialization."""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    else:
        return obj

# Save as JSON (human-readable)
json_file = 'line3_dataset_gt_parameters.json'
with open(json_file, 'w') as f:
    # Convert numpy arrays and types to Python native types for JSON serialization
    json_data = {}
    for exp_id, params in gt_parameters.items():
        json_data[exp_id] = {
            'area_stats': convert_numpy_types(params['area_stats']),
            'optimization_ranges': convert_numpy_types(params['optimization_ranges']),
            'true_pixels_per_mask': convert_numpy_types(params['true_pixels_per_mask'])
        }
    json.dump(json_data, f, indent=2)

print(f"✓ Saved GT parameters to: {json_file}")

# Save as pickle (preserves numpy arrays)
pickle_file = 'line3_dataset_gt_parameters.pkl'
with open(pickle_file, 'wb') as f:
    pickle.dump(gt_parameters, f)

print(f"✓ Saved GT parameters to: {pickle_file}")

# Create a Python file with parameters in the same format as demo_train_CNN_params.py
python_file = 'line3_dataset_gt_params.py'
with open(python_file, 'w') as f:
    f.write("# Line3 Dataset GT Mask Parameters\n")
    f.write("# Generated from ground truth mask analysis\n\n")
    
    # Overall statistics
    if all_true_pixels:
        f.write("# Overall statistics across all mice\n")
        f.write(f"overall_min_area = {overall_min}\n")
        f.write(f"overall_max_area = {overall_max}\n")
        f.write(f"overall_mean_area = {overall_mean}\n")
        f.write(f"overall_median_area = {overall_median}\n")
        f.write(f"overall_std_area = {overall_std}\n")
        f.write(f"total_gt_masks = {len(all_true_pixels)}\n\n")
        
        # Recommended optimization ranges
        f.write("# Recommended optimization ranges based on GT data\n")
        f.write(f"list_minArea = {recommended_minArea}\n")
        f.write(f"list_avgArea = {recommended_avgArea}\n\n")
        
        # Individual mouse statistics
        f.write("# Individual mouse statistics\n")
        f.write("mouse_stats = {\n")
        for exp_id, params in gt_parameters.items():
            stats = params['area_stats']
            f.write(f"    '{exp_id}': {{\n")
            f.write(f"        'min_area': {stats['min_area']},\n")
            f.write(f"        'max_area': {stats['max_area']},\n")
            f.write(f"        'mean_area': {stats['mean_area']},\n")
            f.write(f"        'median_area': {stats['median_area']},\n")
            f.write(f"        'std_area': {stats['std_area']},\n")
            f.write(f"        'total_masks': {stats['total_masks']},\n")
            f.write(f"        'image_dims': '{stats['image_dims']}'\n")
            f.write(f"    }},\n")
        f.write("}\n\n")
        
        # Default parameters (from demo_train_CNN_params.py)
        f.write("# Default parameters (from demo_train_CNN_params.py)\n")
        f.write("thresh_mask = 0.5\n")
        f.write("thresh_COM0 = 2\n")
        f.write("list_thresh_COM = list(np.arange(4, 9, 1))\n")
        f.write("list_thresh_IOU = [0.5]\n")
        f.write("list_cons = list(range(1, 8, 1))\n")

print(f"✓ Saved GT parameters to: {python_file}")

# Display the generated Python file content
print(f"\n{'='*60}")
print(f"Generated Python Parameters File:")
print(f"{'='*60}")
with open(python_file, 'r') as f:
    print(f.read())


✓ Saved GT parameters to: line3_dataset_gt_parameters.json
✓ Saved GT parameters to: line3_dataset_gt_parameters.pkl
✓ Saved GT parameters to: line3_dataset_gt_params.py

Generated Python Parameters File:
# Line3 Dataset GT Mask Parameters
# Generated from ground truth mask analysis

# Overall statistics across all mice
overall_min_area = 10
overall_max_area = 248
overall_mean_area = 43
overall_median_area = 41
overall_std_area = 16
total_gt_masks = 3137

# Recommended optimization ranges based on GT data
list_minArea = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]
list_avgArea = [41]

# Individual mouse statistics
mouse_stats = {
    'mouse6': {
        'min_area': 15,
        'max_area': 79,
        'mean_area': 37,
        'median_area': 36,
        'std_area': 9,
        'total_masks': 968,
        'image_dims': '256x256'
    },
    'mouse7': {
        'min_area': 10,
        'max_area': 197,
        'mean_area': 47,
        'median_area': 47,
        'st